In [1]:
import nltk
from nltk.collocations import *
import pandas as pd

In [2]:
with open('data.txt', 'rb') as f:
    data = f.read()

In [3]:
data = str(data).split(" ")

In [4]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(data)
scored = finder.score_ngrams(bigram_measures.raw_freq)
df = set(bigram for bigram, score in scored)

df = list(df)

num = len(df)
tempword = ''
wtempword = ''
whole = []

for i in range (0, num):
    tempnum = len(df[i])
    for x in range (0, tempnum):
        tempword = df[i][x]
        wtempword = wtempword +  " " + tempword
    whole.append(wtempword.lstrip())
    tempword = ""
    wtempword = ""
    
Resturants =  pd.read_excel(r"(TP) From2016_Sentimental_Burpple.xlsx")
Reviews = Resturants['Cleaned_Reviews'].tolist()
polarity = Resturants['polarity'].tolist()

num = len(whole)
num1 = len(Reviews)
count = 0
countlist = []
targetword = ''
targetreview = ''
average = float(0)
total = float(0)
averagelist = []

for i in range (0, num):
    targetword = whole[i]
    for x in range( 0, num1):
        targetreview = str(Reviews[x])
        if targetreview.find(targetword) >= 0:
            count = count + 1
            total = total + polarity[x]
        targetreview = ""
        
    if count == 0 or total == 0:
        average = 0
    else:
        average = total/count

    averagelist.append(average)
    countlist.append(count)
    count = 0
    average = 0
    total = 0
    targetword = ""
    
Freq1 = pd.DataFrame()

se = pd.Series(whole)
Freq1['Word'] = se

se = pd.Series(countlist)
Freq1['Count'] = se

se = pd.Series(averagelist)
Freq1['Average_Polarity'] = se

import re

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

delList = []
bannedwords = ["tanjong", "singapore" ,"place", "burpple", "price", 
               "address", "duxton", "cafe", "http", "free", "money", 
               "service", "review", "host", "ette", "good", "local", 
               "definitely", "sat", "staff", "sg ", "card", "outram", 
               "robinson", "facebook", "conditioned", "place", "visit", 
               "leong", "good", "unit", "burpple", "bill", "worth",
               "day", "sastisfied", "street", "menu", "well", "low",
               "village", "would", "ayer", "road", "back", "flavour", "try",
               "recommend", "town", "meal", "location", " sg", "sg ", " st",
               "st ", "nice"]
num = len(Freq1)

for i in range (0, num):
    whole[i] = whole[i].replace('\\n','')
    whole[i] = whole[i].replace('\\r','')
    
    if len(whole[i]) <= 3: 
        delList.append(i)
    
    if hasNumbers(whole[i]) == True:
        delList.append(i)
     
    if countlist[i] < 10:
        delList.append(i)
        
    if any(s in whole[i] for s in bannedwords):
        delList.append(i)

### remove dupes

ndelList = set(delList)
delList = list(ndelList)
delList

### deleteing rows

Fin = Freq1.drop(Freq1.index[delList])

Fin = Fin.sort_values(['Count'], ascending=[0])
Fin

In [5]:
ex = pd.ExcelWriter('(TP) Keywords.xlsx', options={'encoding':'utf-8'})
Fin.to_excel(ex, 'Data', index=False)